In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

In [2]:
gfd1 = pd.read_csv("...\\GFD-1\\AssetRollingPerformance.txt", sep="\t", encoding='cp1252')
gfd1.head()

,LipperID,EndDate,ValueLC,BenchmarkValueLC
0,65006437,2017-07-31,3.421730,NaN
1,65006437,2017-06-30,1.536751,NaN
2,65006437,2017-05-31,-1.532318,NaN
3,65006437,2017-04-28,7.743294,NaN
4,65006437,2017-03-31,-0.146545,NaN


In [3]:
cfg = pd.read_csv("...\\CFG\\AssetRollingPerformance.txt", sep="\t")
cfg_cols = cfg.columns
cfg.head()

,LipperID,EndDate,ValueLC,BenchmarkValueLC
0,40000041,2017-07-31,0.627490,NaN
1,40000041,2017-06-30,-0.239290,NaN
2,40000041,2017-05-31,1.418702,NaN
3,40000041,2017-04-28,0.520923,NaN
4,40000041,2017-03-31,0.262723,NaN


In [4]:
cfg.equals(gfd1)

False

In [5]:
print("Missing in GFD-1: " + str(len(cfg[~cfg["LipperID"].isin(gfd1["LipperID"])]["LipperID"])))
missing_in_gfd1_df = cfg[~cfg["LipperID"].isin(gfd1["LipperID"])]
missing_in_gfd1 = cfg[~cfg["LipperID"].isin(gfd1["LipperID"])]["LipperID"]


print("Missing in CFG: " + str(len(gfd1[~gfd1["LipperID"].isin(cfg["LipperID"])]["LipperID"])))
missing_in_cfg_df = gfd1[~gfd1["LipperID"].isin(cfg["LipperID"])]
missing_in_cfg = gfd1[~gfd1["LipperID"].isin(cfg["LipperID"])]["LipperID"]

universe_diffs = pd.concat([missing_in_cfg, missing_in_gfd1])

Missing in GFD-1: 4142
Missing in CFG: 0


In [ ]:
on=["LipperID", "EndDate"]
df = cfg.merge(gfd1, how='outer', on=on, suffixes=("_CFG", "_GFD-1"))
df = df[~df["LipperID"].isin(universe_diffs)]
df = df.fillna("-")
del cfg
del gfd1
df.sample(n=5)

,LipperID,EndDate,ValueLC_CFG,BenchmarkValueLC_CFG,ValueLC_GFD-1,BenchmarkValueLC_GFD-1
2521313,68210065,2014-08-29,3.40265,6.61092,3.40264,6.61092
800819,60098047,2013-04-30,1.6129,1.3775,1.6129,1.3775
2076143,68123507,2016-07-29,2.13927,-,2.13927,-
1142941,65062990,2007-09-28,12.1879,12.945,12.1879,12.945
352254,60040649,2017-02-28,3.57414,-,3.57414,-


In [ ]:
writer = pd.ExcelWriter("Asset Rolling Performance Checks.xlsx", engine='xlsxwriter')
df_dict = {}
missing_in_gfd1_df.to_excel(writer,"Additional in new feed", index=False)
missing_in_cfg_df.to_excel(writer,"Missing in new feed", index=False)
for col in cfg_cols[len(on):]:
    print(col+"\n")
    check=[]
    pc_diff=[]
    columns = []
    columns = on + [col+"_CFG", col+"_GFD-1"]
    #print(columns)
    check_df = df.loc[:, columns]
    for n in range(df.shape[0]):
        try:
            diff = float(round((((check_df.iloc[n, -2])/(check_df.iloc[n, -1])-1)*100),2))
            if ((diff >= -2) and (diff <= 2)):
                sig_diff = False
            else:
                sig_diff = True
        except:
            diff = "-"
            sig_diff = True
        pc_diff.append(diff)
        if str(check_df.iloc[n, -2]) == str(check_df.iloc[n, -1]) or (sig_diff == False):
            check.append("Match")
        elif str(check_df.iloc[n, -2]) == "-":
            check.append("Perf not in CFG")
        elif str(check_df.iloc[n, -1]) == "-":
            check.append("Perf not in GFD-1")
        else:
            check.append("Difference")
    check_df["% Diff"] = np.array(pc_diff)
    check_df["Check"] = np.array(check)
    check_df.to_excel(writer,'%s' % col, index=False)
    display(check_df.sample(n=5))
writer.save()

ValueLC



,LipperID,EndDate,ValueLC_CFG,ValueLC_GFD-1,% Diff,Check
2601751,68230375,2014-12-31,-0.352603,-0.3526,0.0,Match
946170,65019359,2010-08-31,1.31222,1.31221,0.0,Match
3011236,68347752,2017-07-31,1.74807,1.74807,0.0,Match
1020912,65031979,2008-03-31,-3.33516,-3.33515,0.0,Match
2967381,68334513,2015-11-30,0.2224,0.222397,0.0,Match


BenchmarkValueLC



,LipperID,EndDate,BenchmarkValueLC_CFG,BenchmarkValueLC_GFD-1,% Diff,Check
2654792,68240209,2014-01-31,-,-,-,Match
407674,60053026,2013-04-30,1.92666,1.92667,-0.0,Match
523896,60068046,2008-09-30,0.514654,0.514656,-0.0,Match
765157,60094636,2014-07-31,-1.37909,-1.37909,0.0,Match
2051090,68119342,2011-07-29,-2.56498,-2.56497,0.0,Match
